# تحلیل کامنت ها

## ملیکا مرافق 810197581



# مقدمه 
 در این پروژه قصد داریم با استفاده از Naive Bayes Classifier به تحلیل کامنت ها بپردازیم و آن ها را بر اساس محتوا طبقه بندی کنیم. 


# توضیح پروژه
دو مجموعه داده در اختیار داریم که یکی برای آموزش و دیگری برای تست است که هرکدام شامل نظراتی پیرامون تعدادی محصول می باشند.هر نظر یک برچسب دارد که نشانگر این است که محصول پیشنهاد می شود یا خیر.
با کمک Navie Bayes Classifier در داده ی آموزش احتمال حضور هر کلمه در کامنت با برچسب پیشنهاد شده و پیشنهاد نشده را محاسبه می کنیم. در تست قصد داریم با استفاده از این احتمال ها بر اساس محتوای کامنت تصمیم بگیریم که محصول پیشنهاد می شود یا خیر.و در نهایت دقت و درستی مدل خود را می آزماییم.   

In [1]:
from __future__ import unicode_literals
from collections import Counter
from hazm import *
import pandas
import numpy as np


import string
import re
import math

test = pandas.read_csv ('comment_test.csv')
train = pandas.read_csv ('comment_train.csv')


# 1

stemming
دراین روش شناسه و علامت جمع وضمایر متصل و... را از کلمات حذف می کنیم.

lemmatization
در این روش فعل و یا کلمه را به  بن و ریشه اش تبدیل می کنیم.که کتابخانه هضم بن ماضی و مضارع افعال را به دست می آورد.

از آنجاییکه این ریشه ی کلمات است که در معنی آن ها تاثیر می گذارد و شناسه یا تعداد تاثیری بر معنی ان ندارند.همچنین دربرخورد با کلمه ای جدید می توان احتمال آن را برابر با احتمال ریشه اش در صورت وجود قرار داد.   
البته باید در نظر داشت که در این روش با تبدیل شدن فعل منفی به ریشه اش معنی آن تغییر می یابد و در این حالت lemmatization ممکن است باعث کاهش دقت شود. 

In [2]:
def normal(df):

#punctuation
    p = re.compile(r'[^\w\s]+')
    df['comment'] = [p.sub(' ', x) for x in df['comment'].tolist()]
   
#white space new line
    df['comment'] = df['comment'].apply(lambda x: re.sub('\s+',' ',x))


    df['comment'] = df['comment'].apply(lambda x: re.sub("[۰۱۲۳۴۵۶۷۸۹]",' ',x))
   
    df['comment'] = df['comment'].apply(lambda x: re.sub(r'\d+', ' ',x))



    norm = Normalizer()
    df['comment'] = df['comment'].apply(lambda x: norm.normalize(x))


    stop = ['رو','در','این','ان','اون','من', 'برای', 'خودم', 'و','به','با','از','سلام','که','را'] 
    df['comment'] = df['comment'].apply(lambda x: list(item for item in x.split() if item not in stop))



    lem = Lemmatizer()
    df['comment'] = df['comment'].apply(lambda x: set(lem.lemmatize(item) for item in x))
    
   # s = Stemmer()
   # df['comment'] = df['comment'].apply(lambda x: set(s.stem(item) for item in x))
    
    df['comment'] = df['comment'].apply(lambda x: list(x))
    

ستون کامنت و تایتل را یکی کردیم که نسبت به حالت تایتل تنها حدود 4 درصد دقت بالاتری داشت( نسبت به کامنت تنها حدود 2 درصد)

In [3]:

test['comment'] = test['comment'].str.cat(test['title'], sep =" ") 
train['comment'] = train['comment'].str.cat(train['title'], sep = " ")


In [4]:
#normal(train)
#normal(test)


In [5]:
# preprocessing در صورت عدم استفاده  
train['comment'] = train['comment'].apply(lambda x:x.split())
test['comment'] = test['comment'].apply(lambda x:x.split())

# 2



likelihood
احتمال حضور کلمه در یک کلاس: داده آموزش را بر اساس برچسب هر کامنت به دو دسته پیشنهاد شده و پیشنهاد نشده تقسیم می کنیم.سپس تعداد دفعات تکرار آن کلمه(تکرار یک کلمه در کلاس را نادیده میگیریم و یکبار می شماریم) در هر کلاس را بر تعداد کل کامنت های آن کلاس تقسیم می کنیم.

prior
احتمال وقوع هر کلاس(چقدر محتمل است که مستقل از کلمات یک کلاس رخ دهد) در داده آموزش تعداد کامنت هایی را یک برچسب یک کلاس خورده اند را بر کل کامنت ها تقسیم می کنیم

posterior
احتمال وقوع کلاس با داشتن ویژگی ها(کلمات یک کامنت). 
با توجه به کلمات یک کامنت چقدر احتمال دارد که عضو کلاس مورد نظر باشد.


لگاریتم likelihood ها و prior را با هم جمع کردیم.

evidence
چقدر احتمال دارد که یک کلمه در کامنت ظاهر شود( های دو کلاس را با هم جمع کرد likelihood * prior)

# 3

در حالتیکه کلاس c فاقد x باشد P(x|c)  را برابر 0 در نظر می گیرد که با ضرب 0 در باقی احتمال ها P(c|X)برابر 0 می شود و در صورتی که X حاوی کلمه ای نباشد که در'c وجود نداشته باشد  P(c'|X) همیشه بزرگتر از صفر است پس قطعا در 'c قرار می گیرد. 

# 4 
راهکاری برای تخمین داده های دسته ای است.و برای حل مشکل احتمال صفر به کار می رود(که به دلیل عدم وجود یک کلمه در یک یا هردو کلاس ها در داده آموزش رخ می دهد) که مطابق فرمول زیر در این حالت به جای صفر مقدار a/(f+a*d ) 
را می گیرد .مقدار الفا مثبت و کوچکتر مساوی یک است. 



teta = (f + a )/(N + a*d)

In [6]:
def smoothing(f,N):
    a = 0.1
    t = (f + a)/(N + a*d)
    return t

In [7]:

recom = train[train['recommend'] == 'recommended']
not_recom = train[train['recommend'] == 'not_recommended']

In [8]:
recom_cmlist = list(np.concatenate(recom['comment'].tolist()).flat)

notrecom_cmlist = list(np.concatenate(not_recom['comment'].tolist()).flat)

recom_freq = Counter(recom_cmlist)

notrecom_freq = Counter(notrecom_cmlist)


In [9]:
test['comment'] =  [' '.join(map(str, l)) for l in test['comment']]
words = list(test['comment'].str.split(' ', expand=True).stack().unique())

train['comment'] =  [' '.join(map(str, l)) for l in train['comment']]
d = len(list(train['comment'].str.split(' ', expand=True).stack().unique()))


In [10]:
count = train['recommend'].value_counts() 
count

recommended        3000
not_recommended    3000
Name: recommend, dtype: int64

In [11]:

min_val = [0 for i in range(len(words))]

df = {'words': words,'p_recom':min_val,'p_notrecom':min_val,'class':min_val}

ptable = pandas.DataFrame(data=df)


In [12]:
def prob_recom(w):
    if w in recom_freq:
       return math.log(recom_freq[w]/3000) #math.log(smoothing(recom_freq[w],count['recommended'])) #
    else:
        #if w in notrecom_freq:
            return -math.inf # math.log(smoothing(0,count['recommended'])) 
        
def prob_notrecom(w):
    if w in notrecom_freq:
       return math.log(notrecom_freq[w]/3000) #math.log(smoothing(notrecom_freq[w],count['not_recommended'])) #
    else:
        #if w in recom_freq:
             return -math.inf #math.log(smoothing(0,count['not_recommended'])) #

In [13]:

ptable['p_recom'] = ptable['words'].apply(lambda x: prob_recom(x))
ptable['p_notrecom'] = ptable['words'].apply(lambda x: prob_notrecom(x))



In [14]:
def calc_prob(w):

    p = ptable.loc[ptable['words'].isin(w)].sum()
    if p['p_recom'] > p['p_notrecom']:
        return 'recommended'
    else:
        return 'not_recommended'

In [15]:
test['comment'] = test['comment'].apply(lambda x : x.split())
test['class'] = test['comment'].apply(lambda x: calc_prob(x))

In [16]:
test[test['class'] != test['recommend']]

,title,comment,recommend,class
8,نقد پس از خرید,"[سلام, ،, راحت, شدم, از, کابل, شارژ, ،, توصیه,...",recommended,not_recommended
19,خیالم راحت شد,"[فندک, قبلیم, مدام, فیوز, میسوزوند, و, یک, بار...",recommended,not_recommended
23,در یک کلام فوق العاده,"[من, این, روکش, رو, روی, ال, ۹۰, بستم, کیفیتش,...",recommended,not_recommended
34,چیز خوبیه,"[بخرید.چیز, خوبیه.یه, موقع, احتیاج, میشه, بودش...",recommended,not_recommended
47,ظاهری زیبا و باطنی نا ارام و فوق العاده,"[سلام, حدود, یک, ماهه, که, این, هدفون, و, برای...",recommended,not_recommended
67,کاهش زیاد سرعت,"[وحشتناک, سرعتو, کم, میکنه, کاهش, زیاد, سرعت]",not_recommended,recommended
68,زودپز,"[خیلی, خوبه, الان, چند, باری, هست, داریم, استف...",recommended,not_recommended
80,تجربه ای یک ساله,"[با, سلام, خدمت, همه, دوستان, تجربه, یک, ساله,...",recommended,not_recommended
83,چراغ چک موتور خاموش شد,"[سلام, دوستان, بعد, از, استفاده, چراغ, چک, توی...",recommended,not_recommended
101,انقدر که در تصویر دیده می شود زیبا نیست,"[اگر, خریداری, نکنید, بهتره, انقدر, که, در, تص...",not_recommended,recommended


In [17]:
Accuracy = test[test['class'] == test['recommend']].shape[0] / test.shape[0]
Accuracy

0.8625

In [18]:
correct_recommended = test[(test['class'] == test['recommend']) & (test['class'] == 'recommended')].shape[0]
all_recommended = test[test['class'] == 'recommended'].shape[0]

Precision = correct_recommended/all_recommended
Precision

0.9341317365269461

In [19]:
total_recommended = test[test['recommend'] == 'recommended'].shape[0]
Recall = correct_recommended/total_recommended
Recall

0.78

In [20]:
F1 = 2 * Precision * Recall / (Precision + Recall)
F1

0.8501362397820164

# 5

کاربرد Precision زمانی است که false positive ها هزینه ی زیادی داشته باشند(negative هایی که positive تشخیص داده شدند)پس اگر تعداد recommended های تشخیص داده شده کمتر از حالت واقعی باشد(یعنی تعدادی از recommended ها not_recommended تشخیص داده شده باشند;false negative زیاد باشد)به درستی عمل نمی کند.
 
مثلا  در این حالت precison = 1 می باشد:
All detected recommended = 200 - correct detected recommended = 200 - All detected not_recommended = 600 

 
کاربرد Recall زمانی است که false negative ها هزینه ی زیادی داشته باشند(positive هایی که negative تشخیص داده شدند)پس اگر تعداد recommended های تشخیص داده شده بیشتر از حالت واقعی باشد (یعنی تعدادی از not_recommended ها recommended تشخیص داده شده باشند ;false positive ها زیاد باشند)به درستی عمل نمی کند. 
 
مثلا در این حالت  Recall = 1 
correct detected not_recommended = 200 - correct detected recommended = 400 - All detected recommended = 600 (false positive = 200)

# 6

یک میانگین هارمونیک(همساز) است که بین recall و precision تعادل ایجاد می کند و کاربرد آن نسبت به Accurcy زمانی است که false negative و false positive اهمیت بیشتری دارند.
و یا توزیع بین کلاس ها متعادل نیست. 

# 7

|      |  Accuracy  |   Precision  |  Recall  |  F1  |
|------|------------|--------------|----------|------|
|   a  |  93        |   89         |    97    |   93 |
|   b  |    92      |  89          |    97.7  |   93 |
|   c  |  87        |    92        |   81     |  86  |
|   d  |  86        |    93        |      78  |  85  |

# 8
در حالت  a همان طور که مشاهده می کنیم false negative ها حدود 1/5 false positive ها هستند که به همین دلیل recall حدود 8 درصد بهتر از precision عمل کرده و f1 همان طور که گفته شد تعادلی بین recall و precision برقرار کرده.
با توجه به ردیف اول و دوم نتیجه می گیریم که  additive smoothing تاثیر بیشتری به نسبت پیش پردازش دارد. و در ردیف 3 و4 که ازین روش استفاده نکردیم مقدار precision  به مقدار نسبتا زیادی بیشتر از recall شد.
با اعمال پیش پردازش چند کامنت که شامل اعداد بودند و false negative بودند درست تشخیص داده شدند.

# 9

 - با سلام؛ من چندین سال قبل این ادکلن را خریده بودم و بوی بسیار عالی و با ماندگاری خوب بود ولی این بار که از دیجی کالا خریدم اصلا کیفیت گذشته را نداشت!
   236 , recommend = not_recommend

- دستکش کارآمدی نیست با وجودیکه طرح و دوخت زیبایی داره. در صورت استفاده از این دستکش موقع وزنه زدن، بازهم کف دست پینه میبندد
recommend = not_recommend , 231 

- 'برد بالایی نداره کیفیت صداش معمولیه برای تایم تمرین و ورزش مناسبه از روی گوش نمیفته  ظریف نیست میان نمونه های هم رده خودش خیلی خوب نیست.
recommend = not_recommend , 119 

چند کلمه که احتمال حضور آنها در پیشنهاد شده ها بیشتر است مثل زیبا،عالی،کارآمد،ظریف و ... در یک کامنت آمده اند در حالیکه نقیض آنها مد نظر بوده. 

فرض کردیم که کلمات به شرط دانستن کلاس از یکدیگر مستقل هستند که می دانیم در واقعیت اینطور نیست .(برای مثال اگر قبل از نیست چند صفت مثبت بیاید مفهوم منفی است اما مثبت تشخیص داده می شود.همچنین جایگاه کلمات را تاثیر ندادیم و تکرار بیش از یکبار کلمه را در یک کامنت نادبده گرفتیم.


- کاری که میخواین رو انجام میده ولی مایع کیک کمی نشتی میده. title : خیلی خوب
recommend = not_recommend , 439 

باتوجه به تایتل این گمان می رود که اشتباه برچسب خورده است!

- کیفیت کیت و پایه های لامپ های این مهتابی خیلی پایینه . طوریکه هر از گاهی یکی از مهتابی ها از کار میفته ، باید پایه ها رو فشار بدی تا دوباره روشن شه .
  recommend = recommended , 664 
مثل کامنت بالا به نظر می آید اشتباه برچسب خورده است!

 

### منابع 
https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
https://medium.com/syncedreview/applying-multinomial-naive-bayes-to-nlp-problems-a-practical-explanation-4f5271768ebf